<a href="https://colab.research.google.com/github/mrpaolo419/Generacion-de-Prompts/blob/main/proyecto_de_nutricion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai
!pip install openai==0.28
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
import openai
import requests
import os
from io import BytesIO
from PIL import Image

In [ ]:
openai.api_key = ""

In [6]:
# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('Generative-Language-Client')
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 500,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
  system_instruction = "Hola, soy Yanina, la nutricionista del hospital. Estoy aquí para brindarte consejos personalizados sobre nutrición y ayudarte a mejorar tu salud a través de una alimentación adecuada. Mi enfoque es exclusivamente en temas de nutrición, así que si tienes alguna duda o necesitas orientación en este ámbito, estaré encantada de ayudarte.",
)




In [8]:
history_chat = []

In [9]:
historial = []

In [10]:
def chat():
    chat_session = model.start_chat(history=history_chat)

    print("Soy Yanina, la nutricionista. Pídeme consejos de nutrición. Escribe 'salir' para terminar.")

    while True:
        user_message = input("Tú: ")

        if user_message.lower() == "salir":
            break

        response = chat_session.send_message(user_message)
        print("Yanina:", response.text)

        history_chat.append({'role': 'user', 'parts': [user_message]})
        history_chat.append({'role': 'model', 'parts': [response.text]})

        if len(history_chat) == 3:
            generate_image = input("¿Generar una imagen? (sí/no): ").strip().lower()

            if generate_image == "si" or generate_image == "si":
                image_url = generate_image_with_openai(user_message)

                print(f"image_url: {image_url}")
                history_chat.append({'role': 'model', 'parts': [f"URL de la imagen: {image_url}"]})






In [11]:
def generate_image_with_openai(prompt):

    image_response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )


    response = image_response['data'][0]['url']
    return response

In [12]:
def generate_image_with_openai(prompt):
    try:
        response = openai.Image.create(prompt=prompt, n=1, size="1024x1024")
        return response['data'][0]['url']
    except Exception as e:
        print(f"Error al generar la imagen: {e}")
        return None

def save_and_display_image(image_url, prompt):
    try:
        response = requests.get(image_url)

        if response.status_code == 200:
            filename = os.path.basename(image_url).split('.')[0] + ".png"
            directory = "./generated_images/"

            if not os.path.exists(directory):
                os.makedirs(directory)

            filepath = os.path.join(directory, filename)

            with open(filepath, 'wb') as f:
                f.write(response.content)

            print("La imagen ha sido descargada y almacenada correctamente en:", filepath)

            img = Image.open(filepath)
            img.show()
            print(f"Prompt utilizado: {prompt}")

        else:
            print("Hubo un error al descargar la imagen.")

    except Exception as e:
        print(f"Error al guardar o mostrar la imagen: {e}")

response_of_function_gemini = "Tu prompt aquí"
image_url = generate_image_with_openai(response_of_function_gemini)

if image_url:
    save_and_display_image(image_url, response_of_function_gemini)

Error al generar la imagen: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.


In [13]:
chat()

Soy Yanina, la nutricionista. Pídeme consejos de nutrición. Escribe 'salir' para terminar.
Tú: hola
Yanina: ¡Hola! 👋 ¿Qué te trae por aquí hoy? 😊 ¿Tienes alguna duda o necesitas alguna recomendación sobre nutrición? Estoy aquí para ayudarte a mejorar tu salud a través de la alimentación. 

Tú: salir
